## Build Model


In [2]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import joblib
import os
import glob

# Función para cargar imágenes y etiquetas
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in glob.glob(os.path.join(folder, "*.jpg")):
        img = cv2.imread(filename)
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Redimensionar la imagen
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises (opcional)
            img = img.flatten()  # Aplanar la imagen a un vector
            images.append(img)
            labels.append(label)
    return images, labels

# Cargar imágenes y etiquetas
flood_images_path = "C:\\datasets\\flood\\archive\\flood"
fire_images_path = "C:\\datasets\\firesell\\archive\\fire_dataset\\fire_images"
robbery_images_path = "C:\\datasets\\heist\\heavyweaponsguy\\Knife_vs_Pistol\\Knife"

flood_images, flood_labels = load_images_from_folder(flood_images_path, 0)
fire_images, fire_labels = load_images_from_folder(fire_images_path, 1)
robbery_images, robbery_labels = load_images_from_folder(robbery_images_path, 2)

# Combinar todos los datos
images = flood_images + fire_images + robbery_images
labels = flood_labels + fire_labels + robbery_labels

# Convertir a arrays de NumPy
X = np.array(images)
y = np.array(labels)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = SVC(kernel='linear')
model.fit(X_train, y_train)

# Guardar el modelo entrenado
joblib.dump(model, 'event_detection_model.pkl')


['event_detection_model.pkl']

## Detección en tiempo real

In [1]:

import cv2
import joblib
import numpy as np

# Cargar el modelo entrenado
model = joblib.load('event_detection_model.pkl')

# Etiquetas de los eventos
event_labels = {0: 'Inundación', 1: 'Incendio', 2: 'Robo'}

# Capturar video de la cámara
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocesar la imagen de la cámara
    img = cv2.resize(frame, (128, 128))  # Redimensionar la imagen
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises (opcional)
    img = img.flatten()  # Aplanar la imagen a un vector
    img = img.reshape(1, -1)  # Redimensionar para el modelo
    
    # Realizar la predicción
    prediction = model.predict(img)[0]
    
    # Mostrar la predicción en la ventana
    label = event_labels[prediction]
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    # Mostrar la imagen de la cámara
    cv2.imshow('Event Detection', frame)
    
    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()


## Detección a través de samples de imagenes

In [5]:
import cv2
import joblib
import numpy as np

# Cargar el modelo entrenado
model = joblib.load('event_detection_model.pkl')

# Etiquetas de los eventos
event_labels = {0: 'Inundación', 1: 'Incendio', 2: 'Robo'}

# Ruta a la imagen
image_path = './assets/fire/fire.3.png'

# Cargar la imagen
img = cv2.imread(image_path)
if img is None:
    print("Error: no se pudo cargar la imagen.")
else:
    # Preprocesar la imagen
    img_resized = cv2.resize(img, (128, 128))  # Redimensionar la imagen
    img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises (opcional)
    img_flattened = img_gray.flatten()  # Aplanar la imagen a un vector
    img_reshaped = img_flattened.reshape(1, -1)  # Redimensionar para el modelo
    
    # Realizar la predicción
    prediction = model.predict(img_reshaped)[0]
    
    # Obtener la etiqueta correspondiente a la predicción
    label = event_labels[prediction]
    
    # Mostrar la predicción en la imagen
    cv2.putText(img, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    # Mostrar la imagen
    cv2.imshow('Event Detection', img)
    
    # Esperar a que se presione una tecla para cerrar la ventana
    cv2.waitKey(0)
    cv2.destroyAllWindows()


## Usando CNN

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import os
import cv2

# Cargar imágenes y etiquetas
flood_images_path = "C:\\datasets\\flood\\archive\\flood"
fire_images_path = "C:\\datasets\\firesell\\archive\\fire_dataset\\fire_images"
robbery_images_path = "C:\\datasets\\heist\\heavyweaponsguy\\Knife_vs_Pistol\\Knife"

# Función para cargar imágenes y etiquetas
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (128, 128))
            images.append(img)
            labels.append(label)
    return images, labels

# Cargar imágenes y etiquetas
flood_images, flood_labels = load_images_from_folder(flood_images_path, 0)
fire_images, fire_labels = load_images_from_folder(fire_images_path, 1)
robbery_images, robbery_labels = load_images_from_folder(robbery_images_path, 2)

# Combinar todos los datos
images = np.array(flood_images + fire_images + robbery_images)
labels = np.array(flood_labels + fire_labels + robbery_labels)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalizar los valores de los píxeles
X_train = X_train / 255.0
X_test = X_test / 255.0

# Convertir las etiquetas a una representación categórica
y_train = tf.keras.utils.to_categorical(y_train, 3)
y_test = tf.keras.utils.to_categorical(y_test, 3)

# Definir el modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Guardar el modelo entrenado
model.save('cnn_event_detection_model.h5')


c:\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 23s 338ms/step - accuracy: 0.6886 - loss: 0.7532 - val_accuracy: 0.9019 - val_loss: 0.2909
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 303ms/step - accuracy: 0.8972 - loss: 0.3076 - val_accuracy: 0.8992 - val_loss: 0.2450
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 14s 297ms/step - accuracy: 0.9065 - loss: 0.2513 - val_accuracy: 0.8833 - val_loss: 0.3544
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 14s 300ms/step - accuracy: 0.9144 - loss: 0.2209 - val_accuracy: 0.9098 - val_loss: 0.2146
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 303ms/step - accuracy: 0.9326 - loss: 0.1854 - val_accuracy: 0.9178 - val_loss: 0.2397
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 14s 297ms/step - accuracy: 0.9593 - loss: 0.1300 - val_accuracy: 0.8753 - val_loss: 0.4530
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 303ms/step - accuracy: 0.9348 - loss: 0.1817 - val_accuracy: 0.9045 - val_loss: 0.3502
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 305ms/step - accuracy: 0.8346 - loss: 0.4821 - val_accu

## Detección usando CNN


In [10]:
import cv2
import numpy as np
import tensorflow as tf

# Cargar el modelo entrenado
model = tf.keras.models.load_model('cnn_event_detection_model.h5')

# Etiquetas de los eventos
event_labels = {0: 'Inundación', 1: 'Incendio', 2: 'Robo'}

# Ruta a la imagen
image_path = './assets/fire/incendio-casa.jpg'

# Cargar la imagen
img = cv2.imread(image_path)
if img is None:
    print("Error: no se pudo cargar la imagen.")
else:
    # Preprocesar la imagen
    img_resized = cv2.resize(img, (128, 128))  # Redimensionar la imagen
    img_normalized = img_resized / 255.0  # Normalizar los valores de los píxeles
    img_reshaped = img_normalized.reshape(1, 128, 128, 3)  # Redimensionar para el modelo
    
    # Realizar la predicción
    prediction = model.predict(img_reshaped)
    predicted_class = np.argmax(prediction, axis=1)[0]
    
    # Obtener la etiqueta correspondiente a la predicción
    label = event_labels[predicted_class]
    
    # Mostrar la predicción en la imagen
    cv2.putText(img, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    # Mostrar la imagen
    cv2.imshow('Event Detection', img)
    
    # Esperar a que se presione una tecla para cerrar la ventana
    cv2.waitKey(0)
    cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step


## Detección En Tiempo Real Usando CNN

In [4]:
import cv2
import numpy as np
import tensorflow as tf

# Cargar el modelo entrenado
model = tf.keras.models.load_model('cnn_event_detection_model.h5')

# Etiquetas de los eventos
event_labels = {0: 'Inundación', 1: 'Incendio', 2: 'Robo'}

# Capturar video de la cámara
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocesar la imagen de la cámara
    img_resized = cv2.resize(frame, (128, 128))  # Redimensionar la imagen
    img_normalized = img_resized / 255.0  # Normalizar los valores de los píxeles
    img_reshaped = img_normalized.reshape(1, 128, 128, 3)  # Redimensionar para el modelo
    
    # Realizar la predicción
    prediction = model.predict(img_reshaped)
    predicted_class = np.argmax(prediction, axis=1)[0]
    
    # Obtener la etiqueta correspondiente a la predicción
    label = event_labels[predicted_class]
    
    # Mostrar la predicción en la ventana
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    # Mostrar la imagen de la cámara
    cv2.imshow('Event Detection', frame)
    
    # Salir del bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━